## Pięć zasad efektywnej inżynierii promptów
1. Jasne instrukcje poprawiają trafność.
2. Przykłady stabilizują odpowiedź.
3. Zdefiniowany format odpowiedzi = przewidywalny wynik.
4. Dzielenie na kroki = lepsze i pełniejsze rozwiązania.
5. Testy i weryfikacja = bezpieczeństwo i poprawność.

In [7]:
# Import bibliotek
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

# odel bazowy
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

### Zasada 1 — Jasne instrukcje

In [8]:
# Zły prompt — niejasny, bez roli i oczekiwań
bad_prompt = "Napisz funkcję w Pythonie."
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content)

# Dobry prompt — jasno określona rola i oczekiwania
good_prompt = """Jesteś ekspertem programującym w Pythonie.
Napisz funkcję w Pythonie, która przyjmuje listę liczb całkowitych
i zwraca nową listę zawierającą tylko liczby parzyste.
Dodaj test jednostkowy w pytest."""
print("\n=== Dobry prompt ===")
print(llm.invoke(good_prompt).content)

=== Zły prompt ===
Oczywiście! Jaką funkcję chciałbyś, abym napisał? Może to być coś prostego, jak dodawanie dwóch liczb, lub coś bardziej skomplikowanego. Daj mi znać, co dokładnie potrzebujesz!

=== Dobry prompt ===
Oczywiście! Poniżej znajdziesz funkcję w Pythonie, która przyjmuje listę liczb całkowitych i zwraca nową listę zawierającą tylko liczby parzyste. Dodałem również test jednostkowy przy użyciu biblioteki `pytest`.

### Funkcja do filtrowania liczb parzystych

```python
def filtru_parzystych(liczby):
    """Funkcja zwraca listę liczb parzystych z podanej listy liczb całkowitych."""
    return [liczba for liczba in liczby if liczba % 2 == 0]
```

### Test jednostkowy z użyciem pytest

Aby przetestować tę funkcję, możesz stworzyć plik testowy, na przykład `test_filtru_parzystych.py`:

```python
import pytest
from twoj_plik import filtru_parzystych  # Upewnij się, że importujesz funkcję z odpowiedniego pliku

def test_filtru_parzystych():
    assert filtru_parzystych([1, 2, 3, 

### Zasada 2 — Używaj przykładów

In [9]:
# Zadanie: wygeneruj tagi na podstawie treści strony firmy

# Zero-shot
zero_shot = """Podaj tagi opisujące firmę na podstawie tekstu strony:
Firma Lego produkuje zabawki dla dzieci."""
print("=== Zero-shot ===")
print(llm.invoke(zero_shot).content)

# One-shot
one_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.
Przykład:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Teraz:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi:"""
print("\n=== One-shot ===")
print(llm.invoke(one_shot).content)

# Few-shot
few_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.

Przykład 1:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Przykład 2:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi: sport, odzież, obuwie

Teraz:
Tekst: Firma Tesla produkuje samochody elektryczne i magazyny energii.
Tagi:"""
print("\n=== Few-shot ===")
print(llm.invoke(few_shot).content)


=== Zero-shot ===
Oto propozycje tagów opisujących firmę Lego na podstawie podanego tekstu:

- Lego
- zabawki
- dzieci
- produkcja
- kreatywność
- edukacja
- rozrywka
- budowanie
- marka
- innowacja
- gry konstrukcyjne
- rozwój dziecka
- zabawa
- klocki
- jakość

=== One-shot ===
odzież, buty, sport

=== Few-shot ===
samochody, elektryczność, energia


### Zasada 3 — Zdefiniuj format odpowiedzi
walidacja wejścia i wyjścia z wykorzystaniem biblioteki Pydantic

In [1]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import dotenv

dotenv.load_dotenv()

# Definicja schematu wyjścia (Pydantic)
class CityGuide(BaseModel):
    city: str = Field(..., description="Miasto, którego dotyczy przewodnik")
    summary: str = Field(..., description="Krótki opis miasta (2–3 zdania)")
    must_do: List[str] = Field(..., description="Lista 3–5 rzeczy do zrobienia")

# LLM + structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(CityGuide)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem od podróży. Odpowiadaj po polsku, zwięźle."),
    ("user", "Stwórz krótki przewodnik po {city} dla {days}-dniowej wizyty.")
])

chain = prompt | structured_llm

# Walidacja wejścia Pydantic + wywołanie łańcucha
from pydantic import BaseModel, Field

class GuideRequest(BaseModel):
    city: str = Field(min_length=2)
    days: int = Field(ge=1, le=7)

req = GuideRequest(city="Poznań", days=2)

result: CityGuide = chain.invoke(req.model_dump())  # <- dostajesz OBIEKT Pydantic
print(result)                         # CityGuide(city=..., summary=..., must_do=[...])
print(result.model_dump_json(indent=2))  # JSON gotowy do zapisu/transportu

city='Poznań' summary='Poznań to jedno z najstarszych miast w Polsce, znane z bogatej historii, pięknej architektury i tętniącej życiem atmosfery. To idealne miejsce na krótki wypad, oferujące zarówno zabytki, jak i nowoczesne atrakcje.' must_do=['Odwiedź Stary Rynek i zobacz ratusz z koziołkami.', 'Zrelaksuj się w Parku Cytadela, gdzie znajdziesz muzea i piękne tereny zielone.', 'Spróbuj lokalnych specjałów w jednej z restauracji, np. pyry z gzikiem.', 'Zobacz Ostrów Tumski, miejsce narodzin Polski, z katedrą i pięknymi widokami.', 'Zrób zakupy w Starym Browarze, nowoczesnym centrum handlowym w zabytkowym budynku.']
{
  "city": "Poznań",
  "summary": "Poznań to jedno z najstarszych miast w Polsce, znane z bogatej historii, pięknej architektury i tętniącej życiem atmosfery. To idealne miejsce na krótki wypad, oferujące zarówno zabytki, jak i nowoczesne atrakcje.",
  "must_do": [
    "Odwiedź Stary Rynek i zobacz ratusz z koziołkami.",
    "Zrelaksuj się w Parku Cytadela, gdzie znajdzie

### Zasada 4 — Dziel złożone zadania na kroki

In [11]:
# Zły prompt — wszystko naraz
bad_prompt = """Przygotuj trzydniowy plan zwiedzania Poznania z budżetem 300 euro,
uwzględniając atrakcje, restauracje, transport i mapy."""
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content[:600], "...")

# Dobry prompt — krok po kroku
good_step1 = "Wypisz najważniejsze atrakcje kulturalne w Poznaniu z godzinami otwarcia."
step1 = llm.invoke(good_step1).content
print("\n=== Dobry prompt — krok 1 ===")
print(step1[:600], "...")

good_step2 = f"Na podstawie tej listy ułóż plan zwiedzania na 3 dni, maks 4 atrakcje dziennie. Atrakcje: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Dobry prompt — krok 2 ===")
print(step2[:600], "...")


=== Zły prompt ===
Oto trzydniowy plan zwiedzania Poznania z budżetem 300 euro. Plan uwzględnia atrakcje turystyczne, restauracje, transport oraz przybliżone ceny.

### Dzień 1: Stare Miasto i Okolice

**Rano:**
- **Śniadanie:** Kawiarnia "Café La Ruina" (ok. 5 euro)
- **Atrakcja:** Zamek Cesarski (bilet wstępu: 5 euro)
- **Transport:** Spacer do Starego Miasta

**Południe:**
- **Atrakcja:** Rynek i Ratusz (za darmo, obserwacja koziołków o 12:00)
- **Lunch:** Restauracja "Bistro Na Żywo" (ok. 10-15 euro)

**Popołudnie:**
- **Atrakcja:** Katedra na Ostrowie Tumskim (za darmo, opcjonalnie bilet na wieżę: 3 euro)
- ...

=== Dobry prompt — krok 1 ===
Oto niektóre z najważniejszych atrakcji kulturalnych w Poznaniu wraz z godzinami otwarcia. Proszę pamiętać, że godziny mogą się zmieniać, więc warto sprawdzić aktualne informacje przed wizytą.

1. **Stary Rynek** - serce Poznania, gdzie znajduje się ratusz z koziołkami.
   - Godziny otwarcia: Całodobowo (rynek jako przestrzeń publiczna).

2. *

### Zasada 5 — Testuj i weryfikuj wyniki

In [2]:
from langchain.evaluation import load_evaluator
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("embedding_distance", embeddings_model="openai")

result = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="Warszawa jest stolicą Polski"
)

print(result)


{'score': 0.055613485077632974}
